In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import glob
from radprompter import RadPrompter, Prompt, OpenAIClient

In [2]:
sample_report = """
Clinical Information:
67-year-old male with shortness of breath and pleuritic chest pain. Rule out pulmonary embolism.

Technique:
CT pulmonary angiography with IV contrast was performed.

Findings:
There are filling defects within the right and left main pulmonary arteries extending into the lobar branches, compatible with bilateral pulmonary emboli. No evidence of right heart strain.

Ground glass opacities are present within the peripheral lungs bilaterally, suggesting a component of hemorrhage.

The main pulmonary arteries are dilated, with a diameter of 3.4 cm on the right and 3.1 cm on the left (upper limits of normal).

No pleural effusion is identified.

Impression:
1. Bilateral pulmonary emboli involving the right and left main pulmonary arteries and lobar branches.
2. Findings suggestive of pulmonary hemorrhage.
3. Pulmonary artery dilation suggesting chronic pulmonary hypertension.

Recommendation:
Clinical correlation is recommended. Anticoagulation therapy should be initiated if not already started. Consider echocardiography to evaluate for right heart strain.
"""

In [3]:
prompt = Prompt('sample_prompt.toml')

In [4]:
prompt

In [5]:
client = OpenAIClient(
    model = "meta-llama/Meta-Llama-3-8B-Instruct",
    base_url = "http://localhost:9999/v1",
    api_key = "EMPTY",
)

In [6]:
engine = RadPrompter(client, prompt, concurrency=12)

In [7]:
sample = {
    "report": sample_report,
    "hint": prompt.schema[0]['hint'],
    "variable": prompt.schema[0]['variable'],
}
engine(sample)

Processing items: 100%|██████████| 1/1 [00:01<00:00,  1.85s/it]

Present
</initial_answer>
<explanation>
1. The report explicitly mentions "There are filling defects within the right and left main pulmonary arteries extending into the lobar branches, compatible with bilateral pulmonary emboli."
2. This finding is mentioned in the "Findings" section of the report, which suggests that it is a current finding at the time of scan.
3. The report does not mention any previous pulmonary embolism or resolution of a previously observed one.
</explanation>
<final_answer>Present</final_answer>

"Present"



[[{'role': 'system',
   'content': 'You are a helpful assistant that has 20 years of experience in reading radiology reports and extracting data elements.'},
  {'role': 'user',
   'content': 'Carefully review the provided chest CT report (in the <report> tag). Ensure that each data element is accurately captured.\n\nHere is the report:\n<report>\n\nClinical Information:\n67-year-old male with shortness of breath and pleuritic chest pain. Rule out pulmonary embolism.\n\nTechnique:\nCT pulmonary angiography with IV contrast was performed.\n\nFindings:\nThere are filling defects within the right and left main pulmonary arteries extending into the lobar branches, compatible with bilateral pulmonary emboli. No evidence of right heart strain.\n\nGround glass opacities are present within the peripheral lungs bilaterally, suggesting a component of hemorrhage.\n\nThe main pulmonary arteries are dilated, with a diameter of 3.4 cm on the right and 3.1 cm on the left (upper limits of normal).\n\nN